In [22]:
import numpy as np
import pandas as pd
from sklearn import manifold, datasets, decomposition
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import GridSearchCV
from ipywidgets import interact, fixed
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from matplotlib.patches import Ellipse
from scipy import linalg
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, mean_absolute_percentage_error, mean_squared_error
import os
from sklearn.cluster import KMeans
import osmnx as ox
import geopandas as gpd
#import plotly.express as px
import urllib.request
import zipfile
from matplotlib.colors import ListedColormap
import matplotlib.colors as mcolors
from matplotlib.ticker import FuncFormatter
from sklearn.decomposition import PCA, FactorAnalysis
from factor_analyzer import FactorAnalyzer, calculate_kmo, calculate_bartlett_sphericity

In [23]:
osmdf = pd.read_csv('../../data/raw/msa-osm-stats.csv')
msa_emissions = pd.read_csv('../../data/tidy/msa-emissions-by-gas.csv')
msa_acs = pd.read_csv('../../data/tidy/msa-modeshare-employment-salary.csv')
msa_gdp = pd.read_csv('../../data/tidy/msa-gdp.csv')
msa_transit = pd.read_csv('../../data/tidy/msa-transit-vmt-fuel.csv')
msa_transit

,MSA,Diesel (gal),Gasoline (gal),Liquefied Petroleum Gas (gal equivalent),Compressed Natural Gas (gal equivalent),Bio-Diesel (gal),Other Fuel (gal/gal equivalent),Electric Propulsion (kwh),Electric Battery (kwh),Diesel (miles),Gasoline (miles),Liquefied Petroleum Gas (miles),Compressed Natural Gas (miles),Other Fuel (miles),Electric Propulsion (miles),Electric Battery (miles)
0,"Akron, OH",498438,89147,0,849308,0,0,0,0,2619706,560570,0,3166425,0,0,0
1,"Albany, GA",39176,1760,0,147318,0,0,0,0,148053,12499,0,439611,0,0,0
2,"Albany-Schenectady-Troy, NY",2042304,223610,0,0,0,0,0,42126,8632669,1343264,0,0,0,0,0
3,"Albuquerque, NM",1404572,381365,0,224908,792273,0,0,0,6161642,3268665,0,989182,0,0,0
4,"Allentown-Bethlehem-Easton, PA-NJ",677850,126382,0,475045,0,0,0,0,3865849,1128538,0,1603714,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,"Yakima, WA",178977,28063,0,0,0,0,0,0,730634,275527,0,0,0,0,0
254,"York-Hanover, PA",130184,907139,0,179394,0,0,0,0,515734,6299147,0,553503,0,0,0
255,"Youngstown-Warren-Boardman, OH-PA",287458,56240,0,0,0,0,0,0,1467533,486469,0,0,0,0,0
256,"Yuba City, CA",206466,32653,0,0,0,0,0,0,926082,0,0,0,0,0,0


In [24]:
msa_emissions  = msa_emissions.drop(msa_emissions.index[0])

# Merge the DataFrames on the "msa" and "MSA" columns
merged_df = pd.merge(osmdf, msa_emissions , how='inner', left_on='msa', right_on='MSA')
merged_msa = merged_df.drop(columns=['MSA'])

In [25]:
merged_msa.edge_length_total = merged_msa.edge_length_total/1000 #from m to km
merged_msa.edge_length_avg = merged_msa.edge_length_avg/1000 #from m to km
merged_msa.street_length_total = merged_msa.street_length_total/1000 #m -> km
merged_msa.edge_density_km = merged_msa.edge_density_km/1000
merged_msa.street_density_km = merged_msa.street_density_km/1000

osm_stat_dict = {
'n': 'Node count',
'm': 'Edge count',
'k_avg': 'Degree average',
'edge_length_total': 'Edge length total (km)',
'edge_length_avg': 'Edge length average (km)',
'streets_per_node_avg': 'Streets per node average',
'intersection_count': 'Intersection count',
'street_length_total': 'Street length total (km)',
'street_segment_count': 'Street segment count',
'street_length_avg': 'Street length average (km)',
'circuity_avg': 'Circuity average',
'self_loop_proportion': 'Self-loop proportion',
'node_density_km': 'Node density (per sq km)',
'intersection_density_km': 'Intersection density (per sq km)',
'edge_density_km': 'Edge density (km/sq km)',
'street_density_km': 'Street density (km/sq km)',
'builing_area_sqkm': 'Building area (sq km)',
'area_sqkm': 'Area (sq km)',
'Carbon Dioxide':'Carbon Dioxide (Ton)',
'Methane':'Methane (Ton)',
'Nitrous Oxide':'Nitrous Oxide (Ton)'
}

merged_msa=merged_msa.rename(columns=osm_stat_dict)
merged_msa

,msa,Node count,Edge count,Degree average,Edge length total (km),Edge length average (km),Streets per node average,Intersection count,Street length total (km),Street segment count,...,Self-loop proportion,Node density (per sq km),Intersection density (per sq km),Edge density (km/sq km),Street density (km/sq km),Building area (sq km),Area (sq km),Carbon Dioxide (Ton),Methane (Ton),Nitrous Oxide (Ton)
0,"Abilene, TX",14357,39401,5.488751,17168.965935,0.435749,2.881452,11912,8925.436120,20606,...,0.004465,2.012523,1.669790,2.406697,1.251142,7.048505,7133.829924,532679.00283612,27.9820482395,5.0614236844
1,"Albany, GA",8466,22838,5.395228,9545.102272,0.417948,2.837468,6957,4989.592979,11954,...,0.002928,2.028965,1.667318,2.287583,1.195808,6.979580,4172.570196,263831.32031775423,20.83018452565225,4.532439606957801
2,"Akron, OH",26865,70900,5.278243,15191.985931,0.214273,2.756896,21200,8039.294972,36980,...,0.015089,11.231303,8.862967,6.351230,3.360944,14.307097,2391.975398,2007386.7393773547,167.52808604555003,25.02054581165
3,"Albany-Lebanon, OR",7132,17906,5.021312,8280.294430,0.462431,2.568144,5101,4225.438910,9147,...,0.005466,1.193324,0.853498,1.385456,0.706999,7.519705,5976.582645,955233.1995377,98.0240557692,21.2062527474
4,"Albany-Schenectady-Troy, NY",36977,96867,5.239311,27872.441864,0.287739,2.755794,30254,14555.424691,50900,...,0.033320,4.962653,4.060365,3.740738,1.953472,41.455466,7451.054339,878381.4813259091,57.6102337069432,11.738693121229677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,"Worcester, MA-CT",38037,95787,5.036517,24604.031211,0.256862,2.657938,29828,12894.683514,50471,...,0.017416,6.986970,5.479069,4.519485,2.368609,74.415961,5443.990594,2333313.226772765,169.85238166675,31.5916402819
364,"York-Hanover, PA",16622,44476,5.351462,13291.255851,0.298841,2.781194,13507,6831.204674,23064,...,0.006287,7.049714,5.728582,5.637080,2.897247,18.432839,2357.826232,2067360.2114782073,219.9754925947,47.809417099600005
365,"Yuba City, CA",8419,21665,5.146692,7925.868395,0.365837,2.678465,6360,4094.273240,11260,...,0.002220,2.597365,1.962138,2.445227,1.263133,1.594076,3241.362503,288696.43436585995,7.997357688249999,3.84967807065
366,"Youngstown-Warren-Boardman, OH-PA",24885,66650,5.356641,19738.540791,0.296152,2.794736,19982,10381.621775,34674,...,0.006085,5.511548,4.425628,4.371706,2.299329,15.897550,4515.065516,1022715.3651003597,100.22097520127433,15.492663858217666


In [26]:
df = pd.merge(merged_msa, msa_acs , how='inner', left_on='msa', right_on='MSA')


df['msa'] = df['msa'].str.strip()
msa_gdp['MSA'] = msa_gdp['MSA'].str.strip()
final_df = pd.merge(df, msa_gdp , how='inner', left_on='msa', right_on='MSA')
# final_df.drop(columns=['MSA'], inplace=True)
# final_df.to_csv('../../data/tidy/msa-emissions-road-network-socioeconomic-merged.csv', index=False)
# final_df

In [27]:
final_df = pd.merge(final_df, msa_transit , how='outer', left_on='msa', right_on='MSA')

In [28]:
final_df

,msa,Node count,Edge count,Degree average,Edge length total (km),Edge length average (km),Streets per node average,Intersection count,Street length total (km),Street segment count,...,Other Fuel (gal/gal equivalent),Electric Propulsion (kwh),Electric Battery (kwh),Diesel (miles),Gasoline (miles),Liquefied Petroleum Gas (miles),Compressed Natural Gas (miles),Other Fuel (miles),Electric Propulsion (miles),Electric Battery (miles)
0,"Abilene, TX",14357.0,39401.0,5.488751,17168.965935,0.435749,2.881452,11912.0,8925.436120,20606.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Akron, OH",26865.0,70900.0,5.278243,15191.985931,0.214273,2.756896,21200.0,8039.294972,36980.0,...,0.0,0.0,0.0,2619706.0,560570.0,0.0,3166425.0,0.0,0.0,0.0
2,"Albany, GA",8466.0,22838.0,5.395228,9545.102272,0.417948,2.837468,6957.0,4989.592979,11954.0,...,0.0,0.0,0.0,148053.0,12499.0,0.0,439611.0,0.0,0.0,0.0
3,"Albany-Lebanon, OR",7132.0,17906.0,5.021312,8280.294430,0.462431,2.568144,5101.0,4225.438910,9147.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Albany-Schenectady-Troy, NY",36977.0,96867.0,5.239311,27872.441864,0.287739,2.755794,30254.0,14555.424691,50900.0,...,0.0,0.0,42126.0,8632669.0,1343264.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,"Yakima, WA",10920.0,28902.0,5.293407,14155.026874,0.489759,2.709982,8275.0,7249.500452,14773.0,...,0.0,0.0,0.0,730634.0,275527.0,0.0,0.0,0.0,0.0,0.0
366,"York-Hanover, PA",16622.0,44476.0,5.351462,13291.255851,0.298841,2.781194,13507.0,6831.204674,23064.0,...,0.0,0.0,0.0,515734.0,6299147.0,0.0,553503.0,0.0,0.0,0.0
367,"Youngstown-Warren-Boardman, OH-PA",24885.0,66650.0,5.356641,19738.540791,0.296152,2.794736,19982.0,10381.621775,34674.0,...,0.0,0.0,0.0,1467533.0,486469.0,0.0,0.0,0.0,0.0,0.0
368,"Yuba City, CA",8419.0,21665.0,5.146692,7925.868395,0.365837,2.678465,6360.0,4094.273240,11260.0,...,0.0,0.0,0.0,926082.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
final_df = final_df.dropna(subset=['msa'])

In [30]:
final_df = final_df.drop(columns=['MSA_x', 'MSA_y','MSA'])
final_df = final_df.fillna(0)
final_df.to_csv('../../data/tidy/completed-dataset.csv', index=False)
final_df

,msa,Node count,Edge count,Degree average,Edge length total (km),Edge length average (km),Streets per node average,Intersection count,Street length total (km),Street segment count,...,Other Fuel (gal/gal equivalent),Electric Propulsion (kwh),Electric Battery (kwh),Diesel (miles),Gasoline (miles),Liquefied Petroleum Gas (miles),Compressed Natural Gas (miles),Other Fuel (miles),Electric Propulsion (miles),Electric Battery (miles)
0,"Abilene, TX",14357.0,39401.0,5.488751,17168.965935,0.435749,2.881452,11912.0,8925.436120,20606.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Akron, OH",26865.0,70900.0,5.278243,15191.985931,0.214273,2.756896,21200.0,8039.294972,36980.0,...,0.0,0.0,0.0,2619706.0,560570.0,0.0,3166425.0,0.0,0.0,0.0
2,"Albany, GA",8466.0,22838.0,5.395228,9545.102272,0.417948,2.837468,6957.0,4989.592979,11954.0,...,0.0,0.0,0.0,148053.0,12499.0,0.0,439611.0,0.0,0.0,0.0
3,"Albany-Lebanon, OR",7132.0,17906.0,5.021312,8280.294430,0.462431,2.568144,5101.0,4225.438910,9147.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Albany-Schenectady-Troy, NY",36977.0,96867.0,5.239311,27872.441864,0.287739,2.755794,30254.0,14555.424691,50900.0,...,0.0,0.0,42126.0,8632669.0,1343264.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,"Yakima, WA",10920.0,28902.0,5.293407,14155.026874,0.489759,2.709982,8275.0,7249.500452,14773.0,...,0.0,0.0,0.0,730634.0,275527.0,0.0,0.0,0.0,0.0,0.0
366,"York-Hanover, PA",16622.0,44476.0,5.351462,13291.255851,0.298841,2.781194,13507.0,6831.204674,23064.0,...,0.0,0.0,0.0,515734.0,6299147.0,0.0,553503.0,0.0,0.0,0.0
367,"Youngstown-Warren-Boardman, OH-PA",24885.0,66650.0,5.356641,19738.540791,0.296152,2.794736,19982.0,10381.621775,34674.0,...,0.0,0.0,0.0,1467533.0,486469.0,0.0,0.0,0.0,0.0,0.0
368,"Yuba City, CA",8419.0,21665.0,5.146692,7925.868395,0.365837,2.678465,6360.0,4094.273240,11260.0,...,0.0,0.0,0.0,926082.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
final_df.columns

Index(['msa', 'Node count', 'Edge count', 'Degree average',
       'Edge length total (km)', 'Edge length average (km)',
       'Streets per node average', 'Intersection count',
       'Street length total (km)', 'Street segment count',
       'Street length average (km)', 'Circuity average',
       'Self-loop proportion', 'Node density (per sq km)',
       'Intersection density (per sq km)', 'Edge density (km/sq km)',
       'Street density (km/sq km)', 'Building area (sq km)', 'Area (sq km)',
       'Carbon Dioxide (Ton)', 'Methane (Ton)', 'Nitrous Oxide (Ton)',
       '    Car, truck, or van:',
       '    Public transportation (excluding taxicab):', '    Bicycle',
       '    Walked', '    Taxicab, motorcycle, or other means',
       '    Worked from home', 'Median Income', 'Employment Rate',
       'Unemployment Rate', '    Total population', 'Total housing units',
       '        No vehicles available', '        1 vehicle available',
       '        2 vehicles available', '      